## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
#mouse_metadata 249x5

In [2]:
#study_results 1893x4

# surveyed the data sets and see that Mouse ID is that common column in which we'd like to merge 

In [3]:
mouse_study_merge = pd.merge(mouse_metadata, study_results, on="Mouse ID", how="outer")

In [4]:
# Display the data table for preview

In [12]:
mouse_study_merge.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [6]:
# Checking the number of mice.

In [28]:
mice_count_all = mouse_study_merge['Mouse ID'].count()
mice_count_all

1893

In [29]:
mice_count = mouse_study_merge['Mouse ID'].unique()
mice_count_unique = len(mice_count)
mice_count_unique

249

In [47]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
####duplicate_init = mouse_study_merge.groupby(['Mouse ID']).count()['Timepoint']
####duplicate_init

duplicates_init = mouse_study_merge.sort_values('Mouse ID')
#duplicates_init
len(duplicates_init)

1893

In [67]:
timepoint_duplicates = duplicates_init.drop_duplicates(subset=['Mouse ID', 'Timepoint'], keep='first')
len(timepoint_duplicates)

#looks to be successful, as this dropped 5/10 rows -- I'm not sure wheather to pick first, last, keep=False drops all Duplicate rows.

1888

In [73]:
# Optional: Get all the data for the duplicate mouse ID. 

In [88]:
duplicated_timepoints = mouse_study_merge[mouse_study_merge.duplicated()]
duplicated_timepoints

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.0,0


In [93]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.


In [105]:
# Checking the number of mice in the clean DataFrame.
## should be 1888 line items and 249 unique mice here....

In [106]:
clean_data = timepoint_duplicates

In [107]:
len(clean_data)

1888

## Summary Statistics

In [154]:
drug_regimen = clean_data.groupby('Drug Regimen')
#drug_regimen_tumor_vol = drug_regimen.loc['Tumor Volume(mm3)']

drug_regimen
# drug_reg_counts = drug_regimen.count()


In [159]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen


t_vol_mean = drug_regimen['Tumor Volume (mm3)'].mean()
t_vol_median = drug_regimen['Tumor Volume (mm3)'].median()
t_vol_variance = drug_regimen['Tumor Volume (mm3)'].var()
t_vol_std_dev = drug_regimen['Tumor Volume (mm3)'].std()

# This method is the most straighforward, creating multiple series and putting them all together at the end.

t_vol_stats = pd.DataFrame({'Tumor Size Mean': round(t_vol_mean,3), 
                           'Tumor Size Median': round(t_vol_median,3),
                           'Tumor Size Variance': round(t_vol_variance,3),
                           'Tumor Size Std Dev': round(t_vol_std_dev})
t_vol_stats

,Tumor Size Mean,Tumor Size Median,Tumor Size Variance,Tumor Size Std Dev
Drug Regimen,,,,
Capomulin,40.676,41.558,24.948,4.994774
Ceftamin,52.591,51.776,39.290,6.268188
Infubinol,52.885,51.821,43.129,6.567243
Ketapril,55.236,53.699,68.554,8.279709
Naftisol,54.332,52.509,66.173,8.134708
Placebo,54.034,52.289,61.168,7.821003
Propriva,52.389,50.910,43.116,6.566306
Ramicane,40.217,40.673,23.487,4.846308
Stelasyn,54.233,52.432,59.451,7.710419


In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
